6quantile_doublerobust_rbdata: 4791
55improved_result_on_quantile: 5232
4org:  4838

In [12]:
##Normal AR
import numpy as np
import pickle
import math
import sys
sys.path.append("/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl")
def evaluate_policy(model,replay_buffer,env, num_steps=1000,n_eval_episodes=10, deterministic=True,
                    render=False, callback=None, reward_threshold=None,
                    return_episode_rewards=False,seed=None,d=None,noise=None):

    episode_rewards = []
    obs =  env.reset()
    for ep in range(5):
        i=0
        ret=0
        while True:
            obs[d]=obs[d]+np.random.normal(0,noise)
            pi_action = get_action(model,replay_buffer.normalize_obs(obs),True)
            action=pi_action
            obs, rewards, dones, info = env.step(action)
            #obs=agent.Tensor([obs])
            ret += rewards
            i+=1
            if dones:
                obs = env.reset()
                #print("ep:",ep," i",i,ret)
                episode_rewards.append(ret)
                break
    mean_ep_reward=round(np.mean(episode_rewards),1)
    return mean_ep_reward

##########################Evaluate SAC###############################

def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

def get_action(ac,o, deterministic=False):
    a,_=ac(torch.as_tensor(o, dtype=torch.float32, device=device), 1,
                  deterministic) 
    a=a.to('cpu')
    k=a.detach().numpy()
    #print(k[0])
    return k[0]

for noise in [1,0.8,0.6,0.4,0.2]:
    print("\n\n","#"*20,"Noise=",noise,"#"*20,"\n")
    mer=[]
    for dim in range(17):
        print("\n","@"*10,dim,"@"*10,"\n")
        
        maxx={"seed":None,"R100":0,"numep":0}
        sed=[343,75,69,30,2048,5000,6557,7731,8888,9351]
        summ100ret=0
        summret=0
        for seed in sed:
     
            import gym
            import random
            import numpy as np
            import os
            #import matplotlib.pyplot as plt
            import torch
            from datetime import datetime, timedelta
            import time
            from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor
            #from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            #from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor 
            #name="LunarLanderContinuous-v2"
            name="Walker2d-v2"
            seeding(seed)
            os.environ['PYTHONHASHSEED']=str(seed)
            test_env = gym.make(name)
            obs_dim = test_env.observation_space.shape[0]
            act_dim = test_env.action_space.shape[0]
            act_limit = test_env.action_space.high[0]

            encoder_pop_dim=10
            decoder_pop_dim=10

            neuron_input_memory = []
            input_channel_data_shape = (obs_dim * 10) // 4 + (obs_dim * 10) % 4
            # Set core list in a special format easier for reset layer by layer
            if name != "Ant-v3":
                encoder_core_list = [0 for _ in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [1 for _ in range(3)]
                decoder_core_list = [2 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 128 + 3 + layer * 2 for num in range(256)] for layer in range(2)]
            else:
                encoder_core_list = [num // 222 for num in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [5 for _ in range(3)]
                decoder_core_list = [6 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 32 + 7 + layer * 8 for num in range(256)] 
                                          for layer in range(2)]
            core_list = [encoder_core_list]
            core_list.extend(hidden_layer_core_list)
            core_list.extend([decoder_core_list, bias_core_list])


            device = torch.device("cpu")
            m=0#0th model
            #fold="5improved_result_on_quantile"
            #fold="6quantile_doublerobust_rbdata"
            #fold="11_snn_quantile_selu_gausactor"
            #fold="12_snn_quan_selu_gau_drop"
            #fold="14_snn_quan_adaptive_alpha"
            
            #result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"

            result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"
            test_reward, _ = pickle.load(open(result_dir + '/model' + str(m) + '_test_rewards.p', 'rb'))
            best_epoch_idx = 0
            best_epoch_reward = 0
            for idx in range(20):
                if test_reward[(idx + 1) * 5 - 1] > best_epoch_reward:
                    best_epoch_reward = test_reward[(idx + 1) * 5 - 1]
                    best_epoch_idx = (idx + 1)* 5
            #best_epoch_idx=100
            print("#######use epoch: ",best_epoch_idx)        
            model_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '.pt'
            buffer_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '_mean_var.p'
            encoder_var=0.15
            std=math.sqrt(encoder_var)
            spike_ts=5

            # Replay buffer for running z-score norm
            b_mean_var = pickle.load(open(buffer_dir, "rb"))
            replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=1,
                                        clip_limit=3, norm_update_every=1)
            replay_buffer.mean =b_mean_var[0]
            replay_buffer.var = b_mean_var[1]

            #print("mean, var",b_mean_var[0],b_mean_var[1])
            popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
                                                   decoder_pop_dim, (256, 256),
                                       (-3, 3), std, spike_ts, act_limit, device)

            #popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
            #                                       decoder_pop_dim, (256, 256),
             #                          (-3, 3), std, spike_ts, act_limit, device,train=False)
            popsan.load_state_dict(torch.load(model_dir))

            #env = NormalizedActions(env)
            test_env.seed(seed)
            test_env.action_space.seed(seed)
            test_env.observation_space.seed(seed)
            #model.eval()
            print("#"*50)

            mean_ep_reward=evaluate_policy(popsan, replay_buffer, test_env, num_steps=1000, deterministic=True,seed=seed,d=dim,noise=noise)
            print("seed",seed,"Mean_R",mean_ep_reward)
            if mean_ep_reward>maxx["R100"]:
                maxx["R100"]=mean_ep_reward

                maxx["seed"]=seed
            summret=summret+mean_ep_reward
        print("#"*100,"\n",maxx,"\n","#"*100)
        ################Print AVG on other 10 seeds
        print("sum",summret,"AVG:",summret/len(sed))
        mer.append(summret/len(sed))
    #path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/"
    path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/"+fold+"/"

    k="\n".join(str(item) for item in mer)
    filee=open(path+"/test_result_"+fold+"otherseed_epbest95.csv","a")
    filee.write(str(k)+"\n")
    filee.close()



 #################### Noise= 1 #################### 


 @@@@@@@@@@ 0 @@@@@@@@@@ 

[183.07256244 141.87743548 198.92977775]
[454.99310242 444.51894272 668.45708391]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [2]:
##Normal AR
import numpy as np
import pickle
import math
import sys
sys.path.append("/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl")
def evaluate_policy(model,replay_buffer,env, num_steps=1000,n_eval_episodes=10, deterministic=True,
                    render=False, callback=None, reward_threshold=None,
                    return_episode_rewards=False,seed=None,d=None,noise=None):

    episode_rewards = []
    obs =  env.reset()
    for ep in range(5):
        i=0
        ret=0
        while True:
            obs[d]=obs[d]+np.random.normal(0,noise)
            pi_action = get_action(model,replay_buffer.normalize_obs(obs),True)
            action=pi_action
            obs, rewards, dones, info = env.step(action)
            #obs=agent.Tensor([obs])
            ret += rewards
            i+=1
            if dones:
                obs = env.reset()
                #print("ep:",ep," i",i,ret)
                episode_rewards.append(ret)
                break
    mean_ep_reward=round(np.mean(episode_rewards),1)
    return mean_ep_reward

##########################Evaluate SAC###############################

def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

def get_action(ac,o, deterministic=False):
    a,_=ac(torch.as_tensor(o, dtype=torch.float32, device=device), 1,
                  deterministic) 
    a=a.to('cpu')
    k=a.detach().numpy()
    #print(k[0])
    return k[0]

def bst_epch_qcsane(test_reward):
    
    best_epoch_idx = 0
    best_epoch_reward =0# -np.inf#0#for reacher
    for idx in range(20):
        #print(test_reward[0],"nn",test_reward[(idx + 1) * 5 - 1])

        mx_ac=np.argmax(test_reward[(idx + 1) * 5 - 1])
        #print("ee",test_reward)
        if test_reward[(idx + 1) * 5 - 1][mx_ac] > best_epoch_reward:

            best_epoch_reward = test_reward[(idx + 1) * 5 - 1][mx_ac] 
            best_epoch_idx = (idx + 1)* 5
    return best_epoch_idx

def bst_epch(test_reward):
    best_epoch_idx = 0
    best_epoch_reward = 0#-np.inf#0#for reacher
    for idx in range(20):
    #print(test_reward[0],"nn",test_reward[(idx + 1) * 5 - 1])

        mx_ac=np.argmax(test_reward[(idx + 1) * 5 - 1])
        #print("ee",test_reward)
        if test_reward[(idx + 1) * 5 - 1] > best_epoch_reward:

            best_epoch_reward = test_reward[(idx + 1) * 5 - 1]
            best_epoch_idx = (idx + 1)* 5
    return best_epoch_idx
                    
#env_dim=0
ob_dim={"Reacher-v2": 11,"Swimmer-v2":8,"HalfCheetah-v2":17,"Walker2d-v2":17}  
#name="LunarLanderContinuous-v2"
name="Walker2d-v2"
#name="Reacher-v2"
for noise in [1,0.8,0.6,0.4,0.2]:
    print("\n\n","#"*20,"Noise=",noise,"#"*20,"\n")
    mer=[]
    for dim in range(ob_dim[name]):
        print("\n","@"*10,dim,"@"*10,"\n")
        
        maxx={"seed":None,"R100":0,"numep":0}
        sed=[343,75,69,30,2048,5000,6557,7731,8888,9351]
        summ100ret=0
        summret=0
        for seed in sed:
     
            import gym
            import random
            import numpy as np
            import os
            #import matplotlib.pyplot as plt
            import torch
            from datetime import datetime, timedelta
            import time
            from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor
            #from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            #from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor 

            seeding(seed)
            os.environ['PYTHONHASHSEED']=str(seed)
            test_env = gym.make(name)
            obs_dim = test_env.observation_space.shape[0]
            act_dim = test_env.action_space.shape[0]
            act_limit = test_env.action_space.high[0]

            encoder_pop_dim=10
            decoder_pop_dim=10

            neuron_input_memory = []
            input_channel_data_shape = (obs_dim * 10) // 4 + (obs_dim * 10) % 4
            # Set core list in a special format easier for reset layer by layer
            if name != "Ant-v3":
                encoder_core_list = [0 for _ in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [1 for _ in range(3)]
                decoder_core_list = [2 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 128 + 3 + layer * 2 for num in range(256)] for layer in range(2)]
            else:
                encoder_core_list = [num // 222 for num in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [5 for _ in range(3)]
                decoder_core_list = [6 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 32 + 7 + layer * 8 for num in range(256)] 
                                          for layer in range(2)]
            core_list = [encoder_core_list]
            core_list.extend(hidden_layer_core_list)
            core_list.extend([decoder_core_list, bias_core_list])


            device = torch.device("cpu")
            m=0#0th model
            #fold="5improved_result_on_quantile"
            #fold="6quantile_doublerobust_rbdata"
            #fold="11_snn_quantile_selu_gausactor"
            #fold="12_snn_quan_selu_gau_drop"
            #fold="14_snn_quan_adaptive_alpha"
            #fold="15_snn_selu_quan_many_actor"
            #fold="15_snn_selu_quan_3actor_bst procd_3actr_3quantile"
            #fold="15_2actr_2quantile"
            #fold="15_2actr_4quantile"
            #fold="15with2actorseed100_3quantile"
            #fold="15_3actr_2quantile"
            #fold="15_3actr_4quantile"
            #fold="15_4actr_2quantile"
            #fold="15_4actr_3quantile"
            fold="15_4actr_4quantile"
            #result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"
            fold_env="result_walk/usable/"
            #fold_env="result_half/"
            #fold_env="result_reacher/"
            #fold_env="ressult_Swim/"
            result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/"\
                +fold_env+fold+"/params/spike-sac_sac-popsan-"+name+"-encoder-dim-10-decoder-dim-10"
            test_reward, _ = pickle.load(open(result_dir + '/model' + str(m) + '_test_rewards.p', 'rb'))
            if fold[:2]=="15":
                best_epoch_idx=bst_epch_qcsane(test_reward)
            else:
                best_epoch_idx=bst_epch(test_reward)
                #print(test_reward[(idx + 1) * 5 - 1][mx_ac],best_epoch_reward,best_epoch_idx)
            #best_epoch_idx=100
            print("#######use epoch: ",best_epoch_idx)        
            model_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '.pt'
            buffer_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '_mean_var.p'
            encoder_var=0.15
            std=math.sqrt(encoder_var)
            spike_ts=5

            # Replay buffer for running z-score norm
            b_mean_var = pickle.load(open(buffer_dir, "rb"))
            replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=1,
                                        clip_limit=3, norm_update_every=1)
            replay_buffer.mean =b_mean_var[0]
            replay_buffer.var = b_mean_var[1]

            #print("mean, var",b_mean_var[0],b_mean_var[1])
            popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
                                                   decoder_pop_dim, (256, 256),
                                       (-3, 3), std, spike_ts, act_limit, device)

            #popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
            #                                       decoder_pop_dim, (256, 256),
             #                          (-3, 3), std, spike_ts, act_limit, device,train=False)
                
            popsan.load_state_dict(torch.load(model_dir))
            

            #env = NormalizedActions(env)
            test_env.seed(seed)
            test_env.action_space.seed(seed)
            test_env.observation_space.seed(seed)
            #model.eval()
            print("#"*50)

            mean_ep_reward=evaluate_policy(popsan, replay_buffer, test_env, num_steps=1000, deterministic=True,seed=seed,d=dim,noise=noise)
            print("seed",seed,"Mean_R",mean_ep_reward)
            if mean_ep_reward>maxx["R100"]:
                maxx["R100"]=mean_ep_reward

                maxx["seed"]=seed
            summret=summret+mean_ep_reward
        print("#"*100,"\n",maxx,"\n","#"*100)
        ################Print AVG on other 10 seeds
        print("sum",summret,"AVG:",summret/len(sed))
        mer.append(summret/len(sed))
    #path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/"
    path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/"+fold_env+fold+"/"

    k="\n".join(str(item) for item in mer)
    filee=open(path+"/test_result_"+fold+"otherseed_"+str(best_epoch_idx)+".csv","a")
    filee.write(str(k)+"\n")
    filee.close()



 #################### Noise= 1 #################### 


 @@@@@@@@@@ 0 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): SELU()
  (4): Linear(in_features=256, out_features=6, bias=True)
)
##################################################
seed 343 Mean_R 320.5
#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEnco

seed 9351 Mean_R 897.6
#################################################################################################### 
 {'seed': 7731, 'R100': 1040.4, 'numep': 0} 
 ####################################################################################################
sum 6811.9 AVG: 681.1899999999999

 @@@@@@@@@@ 1 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_fea

seed 9351 Mean_R 351.0
#################################################################################################### 
 {'seed': 9351, 'R100': 351.0, 'numep': 0} 
 ####################################################################################################
sum 2904.2 AVG: 290.41999999999996

 @@@@@@@@@@ 2 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_fea

seed 9351 Mean_R 2970.5
#################################################################################################### 
 {'seed': 9351, 'R100': 2970.5, 'numep': 0} 
 ####################################################################################################
sum 18364.0 AVG: 1836.4

 @@@@@@@@@@ 3 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256

seed 9351 Mean_R 5097.2
#################################################################################################### 
 {'seed': 6557, 'R100': 5180.7, 'numep': 0} 
 ####################################################################################################
sum 49866.399999999994 AVG: 4986.639999999999

 @@@@@@@@@@ 4 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features

...

seed 9351 Mean_R 5476.7
#################################################################################################### 
 {'seed': 69, 'R100': 5493.8, 'numep': 0} 
 ####################################################################################################
sum 54826.1 AVG: 5482.61

 @@@@@@@@@@ 13 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256

seed 9351 Mean_R 5492.4
#################################################################################################### 
 {'seed': 69, 'R100': 5500.2, 'numep': 0} 
 ####################################################################################################
sum 54847.700000000004 AVG: 5484.77

 @@@@@@@@@@ 14 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_f

seed 9351 Mean_R 5466.4
#################################################################################################### 
 {'seed': 69, 'R100': 5500.1, 'numep': 0} 
 ####################################################################################################
sum 54829.09999999999 AVG: 5482.909999999999

 @@@@@@@@@@ 15 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=2

seed 9351 Mean_R 5496.9
#################################################################################################### 
 {'seed': 30, 'R100': 5497.2, 'numep': 0} 
 ####################################################################################################
sum 54879.9 AVG: 5487.99

 @@@@@@@@@@ 16 @@@@@@@@@@ 

#######use epoch:  100
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256

seed 9351 Mean_R 5485.4
#################################################################################################### 
 {'seed': 5000, 'R100': 5495.9, 'numep': 0} 
 ####################################################################################################
sum 54822.3 AVG: 5482.2300000000005


In [9]:
-4>-6

True

In [4]:
'{}_'.format(14)

'14_'

In [5]:
import itertools
itertools.count(1)

count(1)